In [1]:
#COOL DATA STUFF
#print('hello world')

In [2]:

import pandas as pd

In [3]:
df_co2 = pd.read_csv("CO2 Emissions_Canada.csv")

In [46]:
df_co2[(df_co2['Make'] == 'ACURA')&(df_co2['Model'] == 'ILX') ]

,Make,Model,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City (L/100 km),Fuel Consumption Hwy (L/100 km),Fuel Consumption Comb (L/100 km),Fuel Consumption Comb (mpg),CO2 Emissions(g/km)
0,ACURA,ILX,COMPACT,2.0,4,AS5,Z,9.9,6.7,8.5,33,196
1,ACURA,ILX,COMPACT,2.4,4,M6,Z,11.2,7.7,9.6,29,221
1071,ACURA,ILX,COMPACT,2.0,4,AS5,Z,9.7,6.7,8.3,34,191
1072,ACURA,ILX,COMPACT,2.4,4,M6,Z,10.8,7.4,9.3,30,214
2203,ACURA,ILX,COMPACT,2.4,4,AM8,Z,9.3,6.6,8.1,35,189
3313,ACURA,ILX,COMPACT,2.4,4,AM8,Z,9.4,6.8,8.2,34,192
4371,ACURA,ILX,COMPACT,2.4,4,AM8,Z,9.4,6.8,8.2,34,192
5454,ACURA,ILX,COMPACT,2.4,4,AM8,Z,9.9,7.0,8.6,33,199
6503,ACURA,ILX,COMPACT,2.4,4,AM8,Z,9.9,7.0,8.6,33,199


In [50]:
df_co2['Vehicle Class'].value_counts()

SUV - SMALL                 1217
MID-SIZE                    1133
COMPACT                     1022
SUV - STANDARD               735
FULL-SIZE                    639
SUBCOMPACT                   606
PICKUP TRUCK - STANDARD      538
TWO-SEATER                   460
MINICOMPACT                  326
STATION WAGON - SMALL        252
PICKUP TRUCK - SMALL         159
MINIVAN                       80
SPECIAL PURPOSE VEHICLE       77
VAN - PASSENGER               66
STATION WAGON - MID-SIZE      53
VAN - CARGO                   22
Name: Vehicle Class, dtype: int64

In [73]:
df_co2[df_co2['Cylinders'] == 3]['Engine Size(L)'].mean()

1.323157894736842

In [47]:
def find_co2_emis(df_co2, make, model, cylinder, transmission):
    make = make.upper()
    model = model.upper()
    transmission = transmission.lower()
    if len(df_co2[(df_co2['Make'] == make)&(df_co2['Model'] == model) ]) < 1:
        print("Car not found please enter average fuel consumption")
        return
    elif len(df_co2[(df_co2['Make'] == make)&(df_co2['Model'] == model) ]) == 1:
        return df_co2[(df_co2['Make'] == make)&(df_co2['Model'] == model) ]['CO2 Emissions(g/km)']
    else:
        #avg them, or also request cylinder and transmission type to make it even more specific.
        #return df_co2[(df_co2['Make'] == make)&(df_co2['Model'] == model) ]['CO2 Emissions(g/km)'].mean()
        subset = df_co2[(df_co2['Make'] == make)&(df_co2['Model'] == model) &(df_co2['Cylinders'] == cylinder)]
        if transmission == 'automatic':
            return subset[subset['Transmission'].str.match("A")]['CO2 Emissions(g/km)'].mean()
        else:
            return subset[subset['Transmission'].str.match("M")]['CO2 Emissions(g/km)'].mean()
        

In [48]:
find_co2_emis(df_co2, 'Acura', 'ILX', 4, 'manual')

217.5

In [91]:
def predict_emissions(df_co2, fuelType, fuelCombo, cylinders, transmission):
    adj_df = df_co2.copy()
    encodedType = pd.get_dummies(adj_df['Fuel Type'])
    adj_df.drop(['Model','Make','Vehicle Class','Fuel Type'], axis  = 1, inplace = True)
    adj_df = adj_df.join(encodedType)
    adj_df['Transmission'] = (adj_df['Transmission'].str.match("A").astype(int))
    if transmission == 'automatic':
      transmission = 1
    else:
      transmission = 0
    if fuelType == 'regular'

    fuelCity = fuelCombo
    fuelHwy = fuelCombo
    engSize = df_co2[df_co2['Cylinders'] == cylinders]['Engine Size(L)'].mean()
    fuelMPG = int(282.481/fuelCombo)
    ourSer = pd.Series([engSize, cylinders,transmision, fuelCity,fuelHwy, fuelCombo, fuelMPG])
    
    return adj_df



In [89]:
predict_emissions(df_co2, 'X', 12.5, 4, 'manual')

,Engine Size(L),Cylinders,Transmission,Fuel Consumption City (L/100 km),Fuel Consumption Hwy (L/100 km),Fuel Consumption Comb (L/100 km),Fuel Consumption Comb (mpg),CO2 Emissions(g/km),D,E,N,X,Z
0,2.0,4,1,9.9,6.7,8.5,33,196,0,0,0,0,1
1,2.4,4,0,11.2,7.7,9.6,29,221,0,0,0,0,1
2,1.5,4,1,6.0,5.8,5.9,48,136,0,0,0,0,1
3,3.5,6,1,12.7,9.1,11.1,25,255,0,0,0,0,1
4,3.5,6,1,12.1,8.7,10.6,27,244,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7380,2.0,4,1,10.7,7.7,9.4,30,219,0,0,0,0,1
7381,2.0,4,1,11.2,8.3,9.9,29,232,0,0,0,0,1
7382,2.0,4,1,11.7,8.6,10.3,27,240,0,0,0,0,1
7383,2.0,4,1,11.2,8.3,9.9,29,232,0,0,0,0,1


In [62]:
df_co2

,Make,Model,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City (L/100 km),Fuel Consumption Hwy (L/100 km),Fuel Consumption Comb (L/100 km),Fuel Consumption Comb (mpg),CO2 Emissions(g/km)
0,ACURA,ILX,COMPACT,2.0,4,AS5,Z,9.9,6.7,8.5,33,196
1,ACURA,ILX,COMPACT,2.4,4,M6,Z,11.2,7.7,9.6,29,221
2,ACURA,ILX HYBRID,COMPACT,1.5,4,AV7,Z,6.0,5.8,5.9,48,136
3,ACURA,MDX 4WD,SUV - SMALL,3.5,6,AS6,Z,12.7,9.1,11.1,25,255
4,ACURA,RDX AWD,SUV - SMALL,3.5,6,AS6,Z,12.1,8.7,10.6,27,244
...,...,...,...,...,...,...,...,...,...,...,...,...
7380,VOLVO,XC40 T5 AWD,SUV - SMALL,2.0,4,AS8,Z,10.7,7.7,9.4,30,219
7381,VOLVO,XC60 T5 AWD,SUV - SMALL,2.0,4,AS8,Z,11.2,8.3,9.9,29,232
7382,VOLVO,XC60 T6 AWD,SUV - SMALL,2.0,4,AS8,Z,11.7,8.6,10.3,27,240
7383,VOLVO,XC90 T5 AWD,SUV - STANDARD,2.0,4,AS8,Z,11.2,8.3,9.9,29,232
